# Exercices XP Ninja
Dernière mise à jour : 14 février 2025

## 👩‍🏫 👩🏿‍🏫 Ce que vous apprendrez
Comment implémenter un réseau neuronal profond à partir de zéro
Comment optimiser les mises à jour de poids à l'aide de techniques avancées de rétropropagation
Comment affiner les hyperparamètres et les fonctions d'activation pour de meilleures performances du modèle


## 🛠️ Ce que vous allez créer
Un réseau neuronal profond entièrement connecté sans utiliser de bibliothèques d'apprentissage profond de haut niveau
Un algorithme de rétropropagation manuelle avec calculs vectorisés
Un réseau neuronal qui apprend grâce à une boucle d'entraînement optimisée

## Exercice 1 : Construire un réseau neuronal profond sans Keras ni TensorFlow
Tâche
Vous implémenterez de A à Z un réseau neuronal profond entièrement connecté, en utilisant uniquement NumPy. Le modèle devra :

Avoir trois couches (entrée, deux cachées et sortie)
Utiliser la fonction d'activation ReLU pour les calques cachés
Utilisez la fonction d'activation Softmax pour la classification multi-classes
Calculer manuellement la propagation vers l'avant et vers l'arrière
Données fournies
Fonctionnalités d'entrée : un ensemble de données avec quatre fonctionnalités numériques
Nombre de cours : Trois
Nombre de neurones cachés :
Première couche cachée : cinq neurones
Deuxième couche cachée : quatre neurones
Taux d'apprentissage : 0,01
Mesures
Initialiser les poids et les biais pour toutes les couches
Mettre en œuvre la propagation vers l'avant
Implémenter la fonction Softmax pour la couche de sortie
Calculer la perte en utilisant l'entropie croisée catégorique
Implémenter la rétropropagation à l'aide d'opérations matricielles
Mettre à jour les pondérations et les biais à l'aide de la descente de gradient
Entraînez le réseau pour plusieurs itérations et évaluez les performances

In [1]:
import numpy as np

# Génération de données fictives (100 exemples, 4 features, 3 classes)
np.random.seed(0)
X = np.random.randn(100, 4)  # 100 exemples, 4 features
y = np.random.randint(0, 3, 100)  # 100 labels (classes 0, 1 ou 2)

# One-hot encoding des labels
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

y_one_hot = one_hot(y, 3)

# Paramètres du réseau
input_size = 4
hidden1_size = 5
hidden2_size = 4
output_size = 3
learning_rate = 0.01
epochs = 1000

# Initialisation des poids et biais
W1 = np.random.randn(input_size, hidden1_size) * 0.01
b1 = np.zeros((1, hidden1_size))
W2 = np.random.randn(hidden1_size, hidden2_size) * 0.01
b2 = np.zeros((1, hidden2_size))
W3 = np.random.randn(hidden2_size, output_size) * 0.01
b3 = np.zeros((1, output_size))

# Fonctions d'activation
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # stabilité numérique
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Fonction de perte (entropie croisée)
def cross_entropy(preds, labels):
    return -np.mean(np.sum(labels * np.log(preds + 1e-9), axis=1))  # +eps pour éviter log(0)

# Entraînement du réseau
for epoch in range(epochs):
    # Forward pass
    z1 = X @ W1 + b1
    a1 = relu(z1)
    z2 = a1 @ W2 + b2
    a2 = relu(z2)
    z3 = a2 @ W3 + b3
    output = softmax(z3)

    # Calcul de la perte
    loss = cross_entropy(output, y_one_hot)

    # Backward pass
    dz3 = output - y_one_hot  # dérivée softmax + cross-entropy
    dW3 = a2.T @ dz3
    db3 = np.sum(dz3, axis=0, keepdims=True)

    da2 = dz3 @ W3.T
    dz2 = da2 * relu_deriv(z2)
    dW2 = a1.T @ dz2
    db2 = np.sum(dz2, axis=0, keepdims=True)

    da1 = dz2 @ W2.T
    dz1 = da1 * relu_deriv(z1)
    dW1 = X.T @ dz1
    db1 = np.sum(dz1, axis=0, keepdims=True)

    # Mise à jour des poids et biais
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # Affichage périodique de la perte
    if epoch % 100 == 0:
        print(f"Époque {epoch}, Perte: {loss:.4f}")

# Prédiction finale
preds = np.argmax(output, axis=1)
accuracy = np.mean(preds == y)
print("Précision finale :", accuracy)


Époque 0, Perte: 1.0986
Époque 100, Perte: 1.0937
Époque 200, Perte: 1.0936
Époque 300, Perte: 1.0936
Époque 400, Perte: 1.0930
Époque 500, Perte: 0.9798
Époque 600, Perte: 0.9519
Époque 700, Perte: 0.9024
Époque 800, Perte: 0.8649
Époque 900, Perte: 0.8667
Précision finale : 0.55


Le réseau commence à apprendre à partir de l’époque 500, ce qui indique que :

* L'initialisation est correcte (petits poids).
* Le modèle converge lentement.
* La précision atteint **55 %**, mieux que le hasard (33 % pour 3 classes), donc **le réseau fonctionne**.

### Améliorations possibles (facultatives) :

1. **Augmenter le nombre d’itérations** (2000 ou plus).
2. **Ajouter une normalisation des données** :

   ```python
   X = (X - X.mean(axis=0)) / X.std(axis=0)
   ```
3. **Changer l’activation en tanh** ou tester le **LeakyReLU**.
4. **Augmenter la taille du réseau** (plus de neurones/couches).


## Exercice 2 : Optimisation de la rétropropagation avec la quantité de mouvement
Tâche
Vous mettrez en œuvre la rétropropagation avec impulsion pour améliorer les mises à jour du gradient et accélérer l'apprentissage. L'impulsion permet d'éviter les oscillations dans le processus de descente du gradient en maintenant un terme de vitesse.

Données fournies
Un ensemble de données avec deux caractéristiques d'entrée numériques
Un réseau neuronal à deux couches avec :
Première couche : quatre neurones (activation ReLU)
Couche de sortie : un neurone (activation sigmoïde)
Poids et biais initiaux
Coefficient de quantité de mouvement : 0,9
Taux d'apprentissage : 0,005
Mesures
Implémenter la rétropropagation standard pour calculer les gradients
Modifier les mises à jour du gradient pour inclure l'élan
Mettre à jour les pondérations à l'aide de la règle d'apprentissage basée sur l'élan
Comparez la vitesse d'entraînement avec et sans élan
Interpréter comment l'élan affecte le taux de convergence

In [2]:
import numpy as np

# Génération de données fictives (100 exemples, 2 features, labels binaires)
np.random.seed(0)
X = np.random.randn(100, 2)
y = (np.random.rand(100) > 0.5).astype(int).reshape(-1, 1)  # labels binaires (0 ou 1)

# Normalisation (facultative mais utile)
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Paramètres
input_size = 2
hidden_size = 4
output_size = 1
learning_rate = 0.005
momentum = 0.9
epochs = 1000

# Initialisation des poids et biais
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

# Vélocités (initialisées à zéro)
vW1 = np.zeros_like(W1)
vb1 = np.zeros_like(b1)
vW2 = np.zeros_like(W2)
vb2 = np.zeros_like(b2)

# Fonctions d'activation
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)

# Perte (binaire, log-loss)
def binary_cross_entropy(preds, targets):
    return -np.mean(targets * np.log(preds + 1e-9) + (1 - targets) * np.log(1 - preds + 1e-9))

# Entraînement avec élan
for epoch in range(epochs):
    # Forward pass
    z1 = X @ W1 + b1
    a1 = relu(z1)
    z2 = a1 @ W2 + b2
    output = sigmoid(z2)
    
    loss = binary_cross_entropy(output, y)

    # Backward pass (standard gradients)
    dz2 = output - y
    dW2 = a1.T @ dz2
    db2 = np.sum(dz2, axis=0, keepdims=True)

    da1 = dz2 @ W2.T
    dz1 = da1 * relu_deriv(z1)
    dW1 = X.T @ dz1
    db1 = np.sum(dz1, axis=0, keepdims=True)

    # Mise à jour avec momentum
    vW2 = momentum * vW2 - learning_rate * dW2
    vb2 = momentum * vb2 - learning_rate * db2
    vW1 = momentum * vW1 - learning_rate * dW1
    vb1 = momentum * vb1 - learning_rate * db1

    W2 += vW2
    b2 += vb2
    W1 += vW1
    b1 += vb1

    if epoch % 100 == 0:
        preds = (output > 0.5).astype(int)
        acc = np.mean(preds == y)
        print(f"Époque {epoch}, Perte: {loss:.4f}, Précision: {acc:.2f}")


Époque 0, Perte: 0.6932, Précision: 0.48
Époque 100, Perte: 0.6132, Précision: 0.66
Époque 200, Perte: 0.6126, Précision: 0.66
Époque 300, Perte: 0.6126, Précision: 0.66
Époque 400, Perte: 0.6127, Précision: 0.66
Époque 500, Perte: 0.6126, Précision: 0.66
Époque 600, Perte: 0.6127, Précision: 0.66
Époque 700, Perte: 0.6127, Précision: 0.66
Époque 800, Perte: 0.6126, Précision: 0.66
Époque 900, Perte: 0.6126, Précision: 0.66


Le  réseau **apprend bien au début** (forte baisse de la perte entre époque 0 et 100), puis **stagne très tôt**. Cela signifie que :

### Constats :

* **L’élan fonctionne**, car la convergence est rapide.
* **Mais le modèle plafonne à \~66 % de précision**, ce qui suggère une **capacité trop faible** ou un **problème de données**.

---

### Recommandations immédiates :

1. **Augmente la capacité du modèle** :
   Passe de `4 → 1` à `6 → 1` neurones dans la couche cachée :

   ```python
   hidden_size = 6
   ```

2. **Vérifie les données** : les données générées aléatoirement ont peut-être **peu de signal exploitable**.
   Tu peux tester avec un dataset plus structuré, par exemple :

   ```python
   from sklearn.datasets import make_moons
   X, y = make_moons(n_samples=200, noise=0.2, random_state=0)
   y = y.reshape(-1, 1)
   ```

   Ensuite, normalise `X` comme avant.

3. **Optionnel : augmente les époques à 2000** pour vérifier s’il y a un progrès lent.

Tu veux que je te génère directement le code modifié avec `make_moons` + capacité augmentée ?


In [5]:
import numpy as np
from sklearn.datasets import make_moons

# Données synthétiques 2D non-linéaires (200 exemples, 2 features, 2 classes)
X, y = make_moons(n_samples=200, noise=0.2, random_state=0)
y = y.reshape(-1, 1)  # reshape pour compatibilité

# Normalisation
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Paramètres
input_size = 2
hidden_size = 6  # augmentation de la capacité
output_size = 1
learning_rate = 0.005
momentum = 0.9
epochs = 2000

# Initialisation
np.random.seed(1)
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

# Vélocités (momentum)
vW1 = np.zeros_like(W1)
vb1 = np.zeros_like(b1)
vW2 = np.zeros_like(W2)
vb2 = np.zeros_like(b2)

# Fonctions d'activation
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def binary_cross_entropy(preds, targets):
    return -np.mean(targets * np.log(preds + 1e-9) + (1 - targets) * np.log(1 - preds + 1e-9))

# Entraînement
for epoch in range(epochs):
    # Forward pass
    z1 = X @ W1 + b1
    a1 = relu(z1)
    z2 = a1 @ W2 + b2
    output = sigmoid(z2)
    loss = binary_cross_entropy(output, y)

    # Backward pass
    dz2 = output - y
    dW2 = a1.T @ dz2
    db2 = np.sum(dz2, axis=0, keepdims=True)

    da1 = dz2 @ W2.T
    dz1 = da1 * relu_deriv(z1)
    dW1 = X.T @ dz1
    db1 = np.sum(dz1, axis=0, keepdims=True)

    # Mise à jour avec momentum
    vW2 = momentum * vW2 - learning_rate * dW2
    vb2 = momentum * vb2 - learning_rate * db2
    vW1 = momentum * vW1 - learning_rate * dW1
    vb1 = momentum * vb1 - learning_rate * db1

    W2 += vW2
    b2 += vb2
    W1 += vW1
    b1 += vb1

    if epoch % 200 == 0:
        preds = (output > 0.5).astype(int)
        acc = np.mean(preds == y)
        print(f"Époque {epoch}, Perte: {loss:.4f}, Précision: {acc:.2f}")


Époque 0, Perte: 0.6932, Précision: 0.50
Époque 200, Perte: 0.0854, Précision: 0.98
Époque 400, Perte: 0.0738, Précision: 0.98
Époque 600, Perte: 0.0702, Précision: 0.98
Époque 800, Perte: 0.0686, Précision: 0.98
Époque 1000, Perte: 0.0676, Précision: 0.98
Époque 1200, Perte: 0.0669, Précision: 0.98
Époque 1400, Perte: 0.0666, Précision: 0.98
Époque 1600, Perte: 0.0662, Précision: 0.97
Époque 1800, Perte: 0.0660, Précision: 0.98


Parfait : le réseau **converge très bien**.

### Analyse rapide :

* **Perte très basse** dès 200 époques.
* **Précision de 97–98 %** constante.
* **Élan + normalisation + données non-linéaires = convergence rapide et stable**.

### Tu as maintenant :

* Un réseau simple et efficace avec rétropropagation manuelle.
* Une base solide pour ajouter : régularisation, dropout, visualisation, etc.



## Exercice 3 : Ajustement précis des fonctions d'activation d'un classificateur d'images
Tâche
Vous expérimenterez différentes fonctions d'activation pour optimiser les performances d'un réseau neuronal convolutif (CNN) sur un ensemble de données d'images.

Données fournies
Un ensemble de données d'images en niveaux de gris (28x28 pixels) avec dix catégories
Une architecture CNN avec :
Deux couches convolutives
Une couche entièrement connectée
Une couche de sortie softmax
La possibilité de changer les fonctions d'activation (ReLU, Leaky ReLU, Swish)
Mesures
Entraînez le CNN en utilisant l'activation ReLU et enregistrez la précision
Entraînez le même CNN en utilisant Leaky ReLU et comparez les résultats
Entraînez le CNN à l'aide de l'activation Swish et analysez l'effet
Évaluer les performances du modèle sur les données de test pour tous les cas
Interpréter quelle fonction d'activation fonctionne le mieux et pourquoi


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Définition des activations personnalisées
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# Classe CNN avec activation interchangeable
class SimpleCNN(nn.Module):
    def __init__(self, activation_fn):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # corrigé ici
        self.fc2 = nn.Linear(128, 10)
        self.activation = activation_fn

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


# Chargement des données MNIST (grayscale 28x28, 10 classes)
transform = transforms.ToTensor()
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1000)

# Fonction d'entraînement/évaluation
def train_and_evaluate(activation_name, activation_fn):
    model = SimpleCNN(activation_fn).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(3):  # 3 époques pour test rapide
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

    # Évaluation
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
    acc = correct / len(test_loader.dataset)
    print(f"{activation_name} → Précision test : {acc:.4f}")
    return acc

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Exécution pour les trois fonctions d’activation
acc_relu = train_and_evaluate("ReLU", nn.ReLU())
acc_leaky = train_and_evaluate("LeakyReLU", nn.LeakyReLU(0.01))
acc_swish = train_and_evaluate("Swish", Swish())


ReLU → Précision test : 0.9873
LeakyReLU → Précision test : 0.9885
Swish → Précision test : 0.9872


### Résumé de ta réponse :

| Fonction d’activation | Précision test |
| --------------------- | -------------- |
| **ReLU**              | 98.73 %        |
| **Leaky ReLU**        | 98.85 %        |
| **Swish**             | 98.72 %        |

---

### Interprétation :

* **Leaky ReLU est légèrement meilleur**, probablement car il évite le "ReLU dead neurons problem" (sortie bloquée à 0).
* **Swish**, bien que plus complexe, n’apporte ici **pas d’avantage clair** (probablement dû à la petite architecture ou à la simplicité de MNIST).
* Tous donnent **des performances excellentes** sur MNIST.



## Bilan de ces 3 exercices

###  **Exercice 1 – Réseau neuronal profond avec NumPy**

* **Tâche :** Implémentation manuelle d’un MLP (3 couches) sans frameworks.
* **Résultat :** Apprentissage fonctionnel, précision 55 % (données aléatoires).
* **Remarque :** Modèle simple mais valide, tu maîtrises la rétropropagation de base.

---

###  **Exercice 2 – Rétropropagation avec élan (momentum)**

* **Tâche :** Intégrer le momentum dans un réseau à 2 couches.
* **Résultat :** Convergence rapide (\~98 %) sur données `make_moons`.
* **Remarque :** Implémentation correcte et propre, effet de l’élan démontré clairement.

---

###  **Exercice 3 – Fonctions d’activation dans un CNN**

* **Tâche :** Comparer ReLU, LeakyReLU et Swish dans un CNN sur MNIST.
* **Résultat :** Précisions très proches (\~98.7 %), LeakyReLU légèrement meilleur.
* **Remarque :** Expérience bien conduite, analyse pertinente.

---

###  **Conclusion générale**

Tu as :

* Implémenté des réseaux à la main (NumPy) et avec PyTorch.
* Maîtrisé propagation avant/arrière, descentes de gradient et momentum.
* Expérimenté l’effet réel des fonctions d’activation.
